<a href="https://colab.research.google.com/github/iraidaantropova/RS/blob/main/HW4_RS_SinitsaI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Урок 4. Рекомендательные системы на основе контента


1 Изучите структуру модуля src

2 Перенесите функции  prefilter_items и postfilter_items из вебинара в модуль src.utils.py

3 Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в
src.recommenders.py

4 Проверьте, что все модули корректно импортируются

5 Если вы еще не прочитали статью о рекомендательных системах и поиске в hh.ru, то обязательно прочитайте

In [1]:
import pandas as pd
import numpy as np

In [2]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

In [3]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    return hit_rate(recommended_list[:k], bought_list)



In [4]:
def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [5]:
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [6]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    flags = np.isin(recommended_list, bought_list)
    return np.dot(flags, prices_recommended).sum() / prices_recommended.sum()

In [7]:
def recall(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)

In [8]:
def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)

In [9]:
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
    flags = np.isin(recommended_list, bought_list)
    return np.dot(flags, prices_recommended).sum() / prices_bought.sum()


In [10]:
def ap_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    amount_relevant = len(relevant_indexes)

    relevant_indexes = relevant_indexes[relevant_indexes <= k]

    sum_ = sum(
        [precision_at_k(recommended_list, bought_list, k=index_relevant + 1) for index_relevant in relevant_indexes])
    return sum_ / amount_relevant

In [11]:
import os
import inspect
import shutil

In [13]:
# Создаю папку src
src_dir = 'src'
os.makedirs(src_dir, exist_ok=True)

#функции для переноса
functions_to_move = [
    hit_rate,
    hit_rate_at_k,
    precision,
    precision_at_k,
    money_precision_at_k,
    recall,
    recall_at_k,
    money_recall_at_k,
    ap_k
]

In [14]:
# Перенос функций в src.metrics.py
with open(os.path.join(src_dir, 'metrics.py'), 'a', encoding='utf-8') as f:
    for function in functions_to_move:
        function_source = inspect.getsource(function)
        f.write(function_source)
        f.write('\n')

In [16]:
def prefilter_items(data, take_n_popular=5000, item_features=None):
    # Уберу не интересные для рекоммендаций категории (department)
    if item_features is not None:
        department_size = pd.DataFrame(item_features. \
                                       groupby('department')['item_id'].nunique(). \
                                       sort_values(ascending=False)).reset_index()

        department_size.columns = ['department', 'n_items']
        rare_departments = department_size[department_size['n_items'] < 150].department.tolist()
        items_in_rare_departments = item_features[
            item_features['department'].isin(rare_departments)].item_id.unique().tolist()

        data = data[~data['item_id'].isin(items_in_rare_departments)]

    # Уберу самые дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
    data = data[data['price'] > 2]

    # Уберу самые дорогие товарыs
    data = data[data['price'] < 50]

    # Возьму топ по популярности
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

    top = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()

    # Заведу фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
    data.loc[~data['item_id'].isin(top), 'item_id'] = 999999


    return data

def postfilter_items(user_id, recommednations):
    pass

In [23]:
# Переношу функцию prefilter_items в модуль src.utils.py

functions_to_move = [
    prefilter_items
]

# Перенос функций в src.utils.py
with open(os.path.join(src_dir, 'utils.py'), 'a', encoding='utf-8') as f:
    for function in functions_to_move:
        function_source = inspect.getsource(function)
        f.write(function_source)
        f.write('\n')